In [ ]:
import time
import numpy as np
import networkx as nx
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path


In [ ]:
def build_knn_csr(X, k=20):
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(X)
    dists, idxs = nbrs.kneighbors(X)
    n = X.shape[0]
    rows = np.repeat(np.arange(n), k)
    cols = idxs[:,1:].ravel()
    vals = dists[:,1:].ravel()
    return csr_matrix((vals, (rows, cols)), shape=(n,n))

def time_method(A, method="FW"):
    if method == "FW":
        t0 = time.time()
        D = nx.floyd_warshall_numpy(nx.from_scipy_sparse_array(A))
        t1 = time.time()
    else:
        t0 = time.time()
        D = shortest_path(A, directed=False, method=method)  # "D" = Dijkstra, "J" = Johnson
        t1 = time.time()
    return t1 - t0, D


In [ ]:
import matplotlib.pyplot as plt

Ns = [200, 500, 1000, 2000]  # prueba escalando
times_fw = []
times_dijkstra = []

for n in Ns:
    X = np.random.randn(n,3)
    A = build_knn_csr(X, k=20)
    # Floyd–Warshall
    t_fw, _ = time_method(A, method="FW")
    times_fw.append(t_fw)
    # Dijkstra
    t_d, _ = time_method(A, method="D")
    times_dijkstra.append(t_d)
    print(f"N={n} | FW={t_fw:.2f}s | Dijkstra={t_d:.2f}s")

plt.plot(Ns, times_fw, "o-", label="Floyd-Warshall")
plt.plot(Ns, times_dijkstra, "s-", label="Dijkstra (scipy)")
plt.xlabel("Número de nodos (N)")
plt.ylabel("Tiempo (s)")
plt.legend(); plt.grid(True)
plt.title("Benchmark geodésicas")
plt.show()
